Use os dados do arquivo abalone. Faça os pre-processamentos do exercício 3.
Usando um 5-fold externo para calcular a acurácia, e um 3-fold interno para 
a escolha dos hiperparâmetros, determine qual algoritimo entre kNN, SVM com 
kernel RBF, redes neurais, Random Forest, e Gradient Boosting Machine tem a 
maior acurácia. Imprima a acurácia com 3 digitos.

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, binarize
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

def pre_processing_data():
    # lê os dados do csv
    dataset= pd.read_csv(
        '/home/mauricio/projects/MO444/lista 4/abalone.csv').values
    # Primeiro transformo a coluna categorica (M,F,I) em numerica (0,1,2)
    labelEncoder = LabelEncoder()
    # transforma a primeira coluna (M,F,I) em (0,1,2)
    dataset[:,0] = labelEncoder.fit_transform(dataset[:,0])
    # Agora eu aplico o One Hot Coding na primeira coluna
    oneHotEncoder = OneHotEncoder(categorical_features=[0])
    dataset = oneHotEncoder.fit_transform(dataset).toarray()
    # Separo a ultima coluna para poder binarizar
    target = dataset[:,10].reshape(-1,1)
    # Para valores > 13 transformo em 1, para valores <=13 transformo em 0
    binarize(target,threshold=13, copy=False)
    # Para funcionar com o StratifiedKFold
    target.shape = (4176,)
    #Substituo os valores transformados no dataset original
    dataset = np.delete(dataset,-1,1)
    # Standarização
    standard_scaler = StandardScaler()
    dataset = standard_scaler.fit_transform(dataset)
    return dataset, target

X, y = pre_processing_data()
# Imprime somente os 10 primeiros dados
print(X[0:10])

[[-0.67495143 -0.68813926  1.31710822 -1.44900723 -1.43989229 -1.18425209
  -1.23034422 -1.17096695 -1.20532696 -1.21305408]
 [ 1.4815881  -0.68813926 -0.75923905  0.0498915   0.12201495 -0.10824748
  -0.30960118 -0.46361041 -0.35684354 -0.20727719]
 [-0.67495143 -0.68813926  1.31710822 -0.69955786 -0.4322102  -0.34735962
  -0.63792816 -0.64833409 -0.60773918 -0.60240383]
 [-0.67495143  1.45319423 -0.75923905 -1.61555154 -1.5406605  -1.42336423
  -1.27214983 -1.2160215  -1.28743825 -1.32081589]
 [-0.67495143  1.45319423 -0.75923905 -0.82446609 -1.08720356 -1.06469603
  -0.97339267 -0.98399054 -0.94074611 -0.85384805]
 [ 1.4815881  -0.68813926 -0.75923905  0.0498915   0.07163085  0.25042072
  -0.10465173 -0.5514668  -0.35684354  0.65481729]
 [ 1.4815881  -0.68813926 -0.75923905  0.17479973  0.17239906 -0.34735962
  -0.12402506 -0.29465583 -0.28385572  0.15192884]
 [-0.67495143 -0.68813926  1.31710822 -0.40810533 -0.38182609 -0.34735962
  -0.65118359 -0.64382864 -0.62142439 -0.53056262]


Função comum para executar qualquer classificador usando Grid Search

In [2]:
# param_grid hiperparâmetros a serem testados
# classifier Instancia do classificador
# dataset dados
# target Label de classficação 
def nested_cross_validation(param_grid, classifier, dataset, target):
    # O Grid Search já é criado com valores de 3-fold e StratifiedKfold
    # por default
    gridSearch = GridSearchCV(
            estimator = classifier, param_grid = param_grid, cv = 3)
    gridSearch.fit(X, y)
    # Faço a validaçao cruzada com 5-fold
    nested_score = cross_val_score(
        gridSearch, X = dataset, y = target, cv = 5)
    # Retorno o resultado da classificação com a validação cruzada de 
    # 5-fold e também o Objeto para recuperarmos os melhores parâmetros 
    # encontrados pelo Grid Search
    return nested_score, gridSearch 

## KNN
1 - Para o kNN, faça um PCA que mantem 90% da variância. Busque os valores do k 
entre os valores 1, 5, 11, 15, 21, 25.

In [3]:
# PCA para Reduzirmos dimensão dos dados para manter 90% da variância
pca = PCA(n_components=0.90)
pca_X = pca.fit_transform(X)
param_grid_knn = [{'n_neighbors': [ 1, 5, 11, 15, 21, 25]}]
knn = KNeighborsClassifier()
nested_knn, knn_grid = nested_cross_validation(
    param_grid_knn, knn, pca_X, y)
# Média da acurácia
print("Acurácia média do KNN:  %0.3f" %nested_knn.mean())

Acurácia média do KNN:  0.882


## SVM
2 - Para o SVM RBF teste para C=$2^{-5}$, $2^{0}$, $2^{5}$, $2^{10}$ e gamma= $2^{-15}$, $2^{-10}$, $2^{-5}$, $2^{0}$, $2^{5}$.

In [4]:
param_grid_svm = [{'C': [2**(-5), 2**(0), 2**(5), 2**(10)], 
                 'gamma':[ 2**(-15), 2**(-10), 2**(-5), 2**(0), 2**(5)]}]
# default ja é o RBF
svm = SVC(random_state=1)
nested_svm, svm_grid = nested_cross_validation(param_grid_svm, svm, X, y)
# Média da acurácia
print("Acurácia média do SVM:  %0.3f" %nested_svm.mean())

Acurácia média do SVM:  0.892


## Rede Neural MLP
3 - Para a rede neural, teste com 3, 7, 10, e 20 neurônios na camada escondida.

In [5]:
param_grid_mlp_net = [{'hidden_layer_sizes': [(3,), (7,), (10,), (20,)]}]
mlp_net = MLPClassifier(random_state=1)
nested_mlp, mlp_grid = nested_cross_validation(
    param_grid_mlp_net, mlp_net, X, y)
# Média da acurácia
print("Acurácia média da MLP:  %0.3f" %nested_mlp.mean())

/home/mauricio/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)
/home/mauricio/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)
/home/mauricio/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)
/home/mauricio/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)
/home/mauricio/a

Acurácia média da MLP:  0.893


## Random Forest
4 - Para o RF, teste max_features = 2, 3, 5, 7 e n_estimators = 100, 200, 400 e 800.

In [6]:
param_grid_rf = [{'n_estimators': [ 100, 200, 400, 800], 
                 'max_features': [2, 3, 5, 7]}]
random_forest = RandomForestClassifier(random_state=1)
nested_rf, rf_grid = nested_cross_validation(
    param_grid_rf, random_forest, X, y)
# Média da acurácia
print("Acurácia média do Random Forest:  %0.3f" %nested_rf.mean())

Acurácia média do Random Forest:  0.890


## Gradient Boosting Machine
5 - Para o GBM (ou XGB) teste para número de arvores = 30, 70, e 100, com learning 
rate de 0.1 e 0.05, e profundidade da árvore = 5.Você pode tanto usar a versão do 
SKlearn ou o XGBoost.

In [7]:
param_grid_gb = [{'learning_rate':[ 0.1, 0.05], 
                  'n_estimators':[30, 70, 100]}]
gb_machine = GradientBoostingClassifier(random_state=1, max_depth=5)
nested_gb, gb_grid = nested_cross_validation(
    param_grid_gb, gb_machine, X, y)
# Média da acurácia
print("Acurácia média Gradient Boosting Machine: %0.3f" %nested_gb.mean())

Acurácia média Gradient Boosting Machine: 0.888


## Classificador Final
O melhor resultado ficou com as Redes Neurais, apesar de que a diferença ficou muito 
pequena em relação ao SVM. Ao executar a rede neural alguns warnings apareceram

In [8]:
# Recupero os melhores parâmetros encontrados previamente pelo Grid Search 
# com 3-Fold
best_param = mlp_grid.best_params_.get('hidden_layer_sizes')
# Crio o classficador com os melhores parâmetros
final_classifier = MLPClassifier(hidden_layer_sizes=best_param)
# Treino com todos os dados
final_classifier.fit(X,y)
print("Classficador final:\n  %s" %final_classifier)

Classficador final:
  MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(3,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)
